In [4]:
import ORM
import Minio_server
import Minio_and_DHT_logic
import concurrent.futures

In [5]:
def start(ip, port, node_list, access_key, secret_key):
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        server = executor.submit(start_server, port)
    Node_self = Node(ip, port)
    Routing_table = RoutingTable(Node_self)
    for node in node_list:
        Routing_table.add_node(node)
    client = Minio_client(ip, access_key, secret_key, port)

In [12]:
def load_self_file(client, path, file_name):
    client.create_self_bucket(filename[:-4])
    filepath = path+file_name 
    client.put_from_self_bucket(filename[:-4], filepath)
    # + Надо добавить файл с хешем в базу данных
    return True

def load_other_file(client, path, file_name):
    client.create_other_bucket(filename[:-4])
    filepath = path+file_name 
    client.put_from_other_bucket(filename[:-4], filepath)
    return True

def get_file(client, file, bucket_name, file_name, node_list, access_key, secret_key):
    for ip in node_list:
        client.change_other_client(ip, access_key, secret_key)
        context = client.get_from_other_bucket(bucket_name, file_name)
        if context:
            break
    # + Добавить файл и его содержимое в бд
    return context, ip

# Сделать функцию, передающую заголовочные файлы и часть составных файлов
def start_transmitting(file_list, path, client, RoutingTable, access_key, secret_key):
    buckets = RoutingTable.get_buckets()
    if 'header' in file_list[0]:
        for bucket in buckets:
            for node in bucket:
                node_ip = node.get_ip()
                client.change_other_client(node_ip, access_key, secret_key)
                load_other_file(client, path, file_list[0])
    else:
        return "Error! First file must be header!"
    for file in file_list[1:]:
        nearlest = find_closest_nodes(file[:-4])[0]
        client.change_other_client(nearlest[1].get_ip(), access_key, secret_key)
        load_other_file(client, path, file)
    return True
        
# Сделать функцию обращения к начальному узлу
def appeal_to_default_node(client, RoutingTable, access_key, secret_key, bucket, file):
    def_node = RoutingTable.get_default_node()
    client.change_other_client(def_node.get_ip(), access_key, secret_key)
    needed_file = client.get_from_other_bucket(bucket, file)
    return needed_file